In [32]:
import pandas as pd
import re
import numpy as np
df_银行格式 = pd.DataFrame(columns = ['类罪类型',
'反洗钱报告编号',
'场景类型',
'交易ID',
'客户编号',
'交易日期',
'明细来源',
'业务种类',
'业务条线',
'报送业务条线',
'明细序号',
'定活存标志',
'账卡标志',
'客户账号',
'客户名称',
'对公对私标志',
'客户年龄',
'国际卡卡币种',
'币种代码',
'存款种类',
'可售产品编号',
'资金用途代码',
'客户类型代码',
'交易时间',
'交易金额',
'折人民币金额',
'折美元金额',
'借贷标志',
'收付标志',
'交易种类',
'交易方式',
'汇款种类',
'交易渠道种类',
'交易流水号',
'全局事件跟踪号',
'交易柜员号',
'交易代码',
'交易机构',
'交易机构名称',
'交易发生地',
'交易所在国家地区代码',
'交易方向',
'凭证号码',
'凭证种类',
'摘要代码',
'摘要描述',
'账户性质',
'账户类型',
'资金用途',
'开户机构',
'开户机构名称',
'开户机构行政区划',
'反洗钱机构',
'反洗钱机构名称',
'开户日期',
'销户日期',
'上次明细交易日',
'产品种类',
'交易对手系统账号',
'交易对手账号修饰符',
'交易对手账户类型',
'交易对手账户性质',
'交易对手对公对私标志',
'交易对手客户账号',
'交易对手客户编号',
'交易对手客户名称',
'交易对手本外行标识',
'交易对手证件号码',
'交易对手证件类型',
'交易对手证件类型说明',
'交易对手商户编号',
'交易对手商户种类',
'交易对手开户机构',
'交易对手开户机构行政区划',
'交易对手定活存标志',
'交易对手金融机构代码',
'交易对手金融机构名称',
'交易对手金融机构行政区划',
'交易对手金融机构代码网点类型',
'交易对手国籍地区',
'终端信息',
'大额豁免标志',
'老可疑豁免标志',
'新可疑豁免标志',
'代理人姓名',
'代理人证件类型',
'代理人证件类型说明',
'代理人证件号码',
'代理人电话',
'代理人国籍地区',
'是否被代办',
'是否跨境交易',
'结算类型现金转账消费',
'是否跨行交易',
'是否异地交易',
'补录机构',
'账户余额',
'备注1',
'备注2'])
all_流水完整 = pd.read_excel(r'C:\Users\zqy\Documents\WeChat Files\wxid_u74gwfzs932921\FileStorage\File\2022-03\经侦数据demo\4545105520210226153452\账户交易明细表.xlsx',dtype = object)
all_流水完整 = all_流水完整.applymap(lambda x: re.sub('nan','',str(x)))

all_流水完整['交易金额'] = pd.to_numeric(all_流水完整['交易金额'],errors='coerce')  #交易金额指定浮点型
all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: float(x))

print(all_流水完整.dtypes)

all_流水完整 = all_流水完整[all_流水完整['查询反馈结果']=='01']


df_代码 = pd.read_excel(r'K:\代码值.xlsx',dtype = 'object',sheet_name = '代码值（全）')
df_币种 = df_代码[df_代码['值域名称'] =='币种']
df_币种英文码 = df_代码[df_代码['值域名称'] =='币种英文码'][['代码编号','代码名称']]
df_币种英文码.columns = ['英文码','代码名称']
df_代码对应 = pd.merge(df_币种,df_币种英文码,how = 'left',on ='代码名称')
df_流水币种 = pd.merge(all_流水完整,df_代码对应,how = 'left',left_on = '交易币种',right_on = '英文码')
print(df_流水币种)


df_银行格式['客户编号'] = all_流水完整['交易账号']
df_银行格式['交易日期'] = all_流水完整['交易日期'].str[0:10]
df_银行格式['客户账号'] = all_流水完整['交易账卡号']
df_银行格式['客户名称'] = all_流水完整['交易户名']
df_银行格式['对公对私标志'] = all_流水完整['交易户名'].map(lambda x:'0' if len(x)<=3 else ('0' if '·' in x else ('1' if '公司' in x else '')))
df_银行格式['币种代码'] = df_流水币种['代码编号']
df_银行格式['交易时间'] = all_流水完整['交易日期'].str.extract(r'(\d{1,2}):\d{1,2}:\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:(\d{1,2}):\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:\d{1,2}:(\d{1,2})')
df_银行格式['交易时间'] = pd.to_numeric(df_银行格式['交易时间'],errors='coerce').astype(str)+'.00'
df_银行格式['交易金额'] = all_流水完整['交易金额'].astype(float).round(2).astype(str)
df_银行格式['折人民币金额'] = df_银行格式['交易金额'].astype(str)
df_银行格式['借贷标志'] = all_流水完整['收付标志'].map(lambda x:'C' if x=='进' else 'D')
df_银行格式['交易对手客户账号'] = all_流水完整['对手卡号']
df_银行格式['交易对手客户名称'] = all_流水完整['对手户名']
df_银行格式['交易流水号'] = all_流水完整['交易流水号']
df_银行格式['交易柜员号'] = all_流水完整['交易柜员号']
df_银行格式['交易机构名称'] = all_流水完整['交易网点名称']
df_银行格式['摘要描述'] = all_流水完整['摘要说明']
df_银行格式['资金用途'] = all_流水完整['渠道']
df_银行格式['终端信息'] = all_流水完整['IP地址']
df_银行格式['账户余额'] = all_流水完整['交易余额']
df_银行格式['备注1'] = all_流水完整['备注']
df_银行格式['反洗钱报告编号'] = '20220307-01'
df_银行格式['场景类型'] = '场景一'

df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan','',str(x)))
df_银行格式['客户编号'][df_银行格式['客户编号']=='']=df_银行格式['客户账号'][df_银行格式['客户编号']==''] # 用账号填充卡号空白
print(df_银行格式.head(20))


# df_tran = df_银行格式[df_银行格式['对公对私标志']=='1']
df_tran = df_银行格式
df_tran = df_tran.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_tran[0] = df_tran.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_tran[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20220307_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_tran[0])


df_3号令 = pd.DataFrame(columns = ['客户编号','反洗钱报告编号','反洗钱可疑交易笔数','金融机构网点名称','金融机构网点所在地区行政区划代码','反洗钱报告对公对私代码','可疑交易特征描述','反洗钱业务规则名称','反洗钱报告最大交易日期','反洗钱报告最小交易日期','反洗钱报告涉及人民币交易金额','反洗钱报告涉及人民币交易笔数','反洗钱传统渠道人民币交易金额','反洗钱电子银行渠道人民币交易金额','反洗钱新兴电子渠道人民币交易金额','反洗钱其他渠道人民币交易金额','反洗钱传统渠道交易笔数','反洗钱电子银行渠道交易笔数','反洗钱新兴电子渠道交易笔数','反洗钱其他渠道交易笔数','可疑交易措施描述','可疑交易报告报送方向代码','反洗钱可疑行为描述','#报告报送状态代码','报送日期','可疑交易报送标志','#批量导入日期','反洗钱交易主体账户个数','反洗钱中心机构编号','反洗钱中心机构中文名称','#备注二(洗钱类型)','报送次数','接续报告参考标志(0-否 1-是)','非可疑行为描述','可疑交易报告疑点分析说明','可疑交易报告疑点分析及排除理由','反洗钱可疑行为描述2','可疑交易报告疑点分析说明2','风险等级'])
df_3号令['客户编号'] = df_tran.groupby(['客户编号']).size().reset_index()['客户编号']
df_3号令['反洗钱报告对公对私代码'] = df_tran.groupby(['客户编号','对公对私标志']).size().reset_index()['对公对私标志']
df_3号令['#报告报送状态代码'] = '06'
df_3号令 = df_3号令.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_3号令[0] = df_3号令.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_3号令[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_AML_SSPCS_RPT_20220307_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_3号令[0])


交易账卡号        object
交易户名         object
交易账号         object
交易证件号码       object
交易日期         object
交易金额        float64
交易余额        float64
收付标志         object
对手账号         object
对手卡号         object
现金标志         object
对手户名         object
对手身份证号       object
对手开户银行       object
摘要说明         object
交易币种         object
交易网点名称       object
交易场所         object
交易发生地        object
交易是否成功       object
传票号          object
IP地址         object
MAC地址        object
对手交易余额       object
交易流水号        object
对手余额         object
渠道           object
日志号          object
凭证种类         object
凭证号          object
交易柜员号        object
备注           object
查询反馈结果       object
查询反馈结果原因     object
dtype: object


In [30]:
df_1 = all_流水完整
df_1

,交易账卡号,交易户名,交易账号,交易证件号码,交易日期,交易金额,交易余额,收付标志,对手账号,对手卡号,...,交易流水号,对手余额,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,查询反馈结果,查询反馈结果原因
0,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:58:50,5000,9199.65,进,8110201013500078335,,...,SWHZ2016081200010832646626,,,J0000009952984,,0,8195,隋乙锋支付宝转账,01,
1,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:58:05,4500,4199.65,出,7331710182400002709,,...,S5TF2016081200000061826815,,,J0000009949332,,0,0279,,01,
2,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:41:38,2000,8699.65,出,7440810182200001028,,...,S5TF2016081200000061796710,,,J0000009870665,,0,0279,,01,
3,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:40:07,10000,10699.65,出,7331710182400002709,,...,S5TF2016081200000061793922,,,J0000009863225,,0,0279,,01,
4,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:39:03,20000,20699.65,进,8110201013500078335,,...,SWHZ2016081200010832634994,,,J0000009858070,,0,8195,隋乙锋支付宝转账,01,
5,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:22:19,9000,699.65,出,7331710182400002709,,...,S5TF2016081200000061760190,,,J0000009771692,,0,0279,,01,
6,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:17:15,21000,9699.65,出,7331710182400002709,,...,S5TF2016081200000061750290,,,J0000009744293,,0,0279,,01,
7,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 17:04:04,30000,30699.65,进,6216601700000066435,,...,SOBS2016081200000081394243,,,J0000009669721,,0,9002,,01,
8,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 15:48:02,380,699.65,出,7331710182400002709,,...,S5TF2016081200000061582485,,,J0000009203630,,0,0279,,01,
9,6226902704263609,隋乙锋,6226902704263609,232302199003287014,2016-08-12 15:43:39,1000,1079.65,进,8110201013500078335,,...,SWHZ2016081200010832574740,,,J0000009176883,,0,8195,隋乙锋支付宝转账,01,


In [10]:
all_流水完整 = all_流水完整[all_流水完整['查询反馈结果']=='01']
all_流水完整.shape

(19992, 34)

In [42]:


df_代码 = pd.read_excel(r'K:\代码值.xlsx',dtype = 'object',sheet_name = '代码值（全）')
df_币种 = df_代码[df_代码['值域名称'] =='币种']
df_币种英文码 = df_代码[df_代码['值域名称'] =='币种英文码'][['代码编号','代码名称']]
df_币种英文码.columns = ['英文码','代码名称']
df_代码对应 = pd.merge(df_币种,df_币种英文码,how = 'left',on ='代码名称')
df_流水币种 = pd.merge(all_流水完整,df_代码对应,how = 'left',left_on = '交易币种',right_on = '英文码')
print(df_流水币种)


df_银行格式['客户编号'] = all_流水完整['交易账号']
df_银行格式['交易日期'] = all_流水完整['交易日期'].str[0:10]
df_银行格式['客户账号'] = all_流水完整['交易账卡号']
df_银行格式['客户名称'] = all_流水完整['交易户名']
df_银行格式['对公对私标志'] = all_流水完整['交易户名'].map(lambda x:'0' if len(x)<=3 else ('0' if '·' in x else ('1' if '公司' in x else '')))
df_银行格式['币种代码'] = df_流水币种['代码编号']
df_银行格式['交易时间'] = all_流水完整['交易日期'].str.extract(r'(\d{1,2}):\d{1,2}:\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:(\d{1,2}):\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:\d{1,2}:(\d{1,2})')
df_银行格式['交易时间'] = pd.to_numeric(df_银行格式['交易时间'],errors='coerce').astype(str)+'.00'
df_银行格式['交易金额'] = all_流水完整['交易金额'].astype(float).round(2).astype(str)
df_银行格式['折人民币金额'] = df_银行格式['交易金额'].astype(str)
df_银行格式['借贷标志'] = all_流水完整['收付标志'].map(lambda x:'C' if x=='进' else 'D')
df_银行格式['交易对手客户账号'] = all_流水完整['对手卡号']
df_银行格式['交易对手客户名称'] = all_流水完整['对手户名']
df_银行格式['交易流水号'] = all_流水完整['交易流水号']
df_银行格式['交易柜员号'] = all_流水完整['交易柜员号']
df_银行格式['交易机构名称'] = all_流水完整['交易网点名称']
df_银行格式['摘要描述'] = all_流水完整['摘要说明']
df_银行格式['资金用途'] = all_流水完整['渠道']
df_银行格式['终端信息'] = all_流水完整['IP地址']
df_银行格式['账户余额'] = all_流水完整['交易余额']
df_银行格式['备注1'] = all_流水完整['备注']
df_银行格式['反洗钱报告编号'] = '20220307-01'
df_银行格式['场景类型'] = '场景一'

df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan','',str(x)))
df_银行格式['客户编号'][df_银行格式['客户编号']=='']=df_银行格式['客户账号'][df_银行格式['客户编号']==''] # 用账号填充卡号空白
print(df_银行格式.head(20))


# df_tran = df_银行格式[df_银行格式['对公对私标志']=='1']
df_tran = df_银行格式
df_tran = df_tran.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_tran[0] = df_tran.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_tran[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20220307_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_tran[0])


df_3号令 = pd.DataFrame(columns = ['客户编号','反洗钱报告编号','反洗钱可疑交易笔数','金融机构网点名称','金融机构网点所在地区行政区划代码','反洗钱报告对公对私代码','可疑交易特征描述','反洗钱业务规则名称','反洗钱报告最大交易日期','反洗钱报告最小交易日期','反洗钱报告涉及人民币交易金额','反洗钱报告涉及人民币交易笔数','反洗钱传统渠道人民币交易金额','反洗钱电子银行渠道人民币交易金额','反洗钱新兴电子渠道人民币交易金额','反洗钱其他渠道人民币交易金额','反洗钱传统渠道交易笔数','反洗钱电子银行渠道交易笔数','反洗钱新兴电子渠道交易笔数','反洗钱其他渠道交易笔数','可疑交易措施描述','可疑交易报告报送方向代码','反洗钱可疑行为描述','#报告报送状态代码','报送日期','可疑交易报送标志','#批量导入日期','反洗钱交易主体账户个数','反洗钱中心机构编号','反洗钱中心机构中文名称','#备注二(洗钱类型)','报送次数','接续报告参考标志(0-否 1-是)','非可疑行为描述','可疑交易报告疑点分析说明','可疑交易报告疑点分析及排除理由','反洗钱可疑行为描述2','可疑交易报告疑点分析说明2','风险等级'])
df_3号令['客户编号'] = df_tran.groupby(['客户编号']).size().reset_index()['客户编号']
df_3号令['反洗钱报告对公对私代码'] = df_tran.groupby(['客户编号','对公对私标志']).size().reset_index()['对公对私标志']
df_3号令['#报告报送状态代码'] = '06'
df_3号令 = df_3号令.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_3号令[0] = df_3号令.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_3号令[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_AML_SSPCS_RPT_20220307_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_3号令[0])


                  交易账卡号 交易户名              交易账号              交易证件号码  \
0      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
1      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
2      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
3      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
4      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
5      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
6      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
7      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
8      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
9      6226902704263609  隋乙锋  6226902704263609  232302199003287014   
10     6226902704263609  隋乙锋  6226902704263609  232302199003287014   
11     6226902704263609  隋乙锋  6226902704263609  232302199003287014   
12     6226902704263609  隋乙锋  6226902704263609  232302199003287014   
13     6226902704263

   类罪类型      反洗钱报告编号 场景类型 交易ID              客户编号        交易日期 明细来源 业务种类 业务条线  \
0        20220307-01  场景一       6226902704263609  2016-08-12                  
1        20220307-01  场景一       6226902704263609  2016-08-12                  
2        20220307-01  场景一       6226902704263609  2016-08-12                  
3        20220307-01  场景一       6226902704263609  2016-08-12                  
4        20220307-01  场景一       6226902704263609  2016-08-12                  
5        20220307-01  场景一       6226902704263609  2016-08-12                  
6        20220307-01  场景一       6226902704263609  2016-08-12                  
7        20220307-01  场景一       6226902704263609  2016-08-12                  
8        20220307-01  场景一       6226902704263609  2016-08-12                  
9        20220307-01  场景一       6226902704263609  2016-08-12                  
10       20220307-01  场景一       6226902704263609  2016-08-12                  
11       20220307-01  场景一       6226902704263609  20

In [6]:
all_流水完整.groupby(['查询反馈结果','查询反馈结果原因']).size().reset_index()

,查询反馈结果,查询反馈结果原因,0
0,01,,13302
1,01,成功,6690
2,02,未找到符合请求条件的数据GFM0142已换卡，新卡卡号为 6217856000067934256,1
3,02,未找到符合请求条件的数据GFM0144已销卡，查询卡片关联账户失败,2
4,02,未找到符合请求条件的数据GFM0187请于营业时间内办理 BA2754请于营业时间内办理 |...,2
5,02,未查询到交易明细,3


In [13]:
all_流水完整.groupby(['交易币种']).size().reset_index()

,交易币种,0
0,,2
1,CNY,19982
2,USD,8


In [18]:
all_流水完整['交易日期']

0        2016-08-12 17:58:50
1        2016-08-12 17:58:05
2        2016-08-12 17:41:38
3        2016-08-12 17:40:07
4        2016-08-12 17:39:03
5        2016-08-12 17:22:19
6        2016-08-12 17:17:15
7        2016-08-12 17:04:04
8        2016-08-12 15:48:02
9        2016-08-12 15:43:39
10       2016-08-12 09:57:49
11       2016-08-12 09:49:16
12       2016-08-11 20:42:07
13       2016-08-11 20:13:52
14       2016-08-11 20:13:12
15       2016-08-11 19:47:11
16       2016-08-11 19:43:40
17       2016-08-17 14:36:25
18       2016-08-17 09:05:17
19       2016-08-17 02:57:34
20       2016-08-16 23:57:27
21       2016-08-16 23:51:13
22       2016-08-16 17:41:10
23       2016-08-16 16:29:50
24       2016-08-16 16:28:21
25       2016-08-16 14:37:46
26       2016-08-16 14:33:54
27       2016-08-16 13:36:28
28       2016-08-16 13:32:31
29       2016-08-16 13:31:54
                ...         
19970    2020-05-24 00:41:01
19971    2020-05-23 22:06:39
19972    2020-05-23 18:52:16
19973    2019-

In [1]:
all_流水完整.iloc[400]

NameError: name 'all_流水完整' is not defined